In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, BaggingClassifier 
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error

Exercice sur le Random Forest avec Bagging/Pasting
Objectif :

    Mettre en place un modèle de Random Forest pour prédire le prix (target de régression) des vins.
    Mettre en place un modèle de Random Forest pour classifier les vins en fonction de leur variété.

Étapes :

    Nettoyage des données :
        Vérifiez s'il y a des valeurs manquantes dans le dataset et traitez-les.
        Convertissez les variables catégorielles en variables numériques si nécessaire.
        Séparez le dataset en features (X) et la target variable (y) pour la régression et la classification.

    Random Forest pour la Régression (Prédiction de Prix) :
        Utilisez la bibliothèque scikit-learn en Python.
        Divisez le dataset en ensembles d'entraînement (training set) et de test (test set).
        Entraînez un modèle de Random Forest pour prédire le prix des vins.
        Évaluez les performances du modèle en utilisant des métriques appropriées pour la régression (par exemple, MSE, RMSE).

    Random Forest pour la Classification (Variété de Vin) :
        Convertissez la variable de variété en une variable numérique si nécessaire.
        Divisez le dataset en ensembles d'entraînement (training set) et de test (test set).
        Entraînez un modèle de Random Forest pour classifier la variété de vin.
        Évaluez les performances du modèle en utilisant des métriques appropriées pour la classification (par exemple, précision, rappel, F1-score).

    Bagging ou Pasting :
        Comparez les performances du Random Forest avec bagging et du Random Forest avec pasting.
        Utilisez différentes valeurs pour le nombre d'estimateurs (arbres) et observez comment cela affecte les performances.
        Choisissez une métrique d'évaluation appropriée pour comparer les deux approches.

    Analyse des Résultats :
        Analysez et interprétez les résultats obtenus.
        Identifiez les hyperparamètres optimaux pour chaque modèle.

In [9]:
# Chargement du dataset
url = "datasets/wine.csv" 
data = pd.read_csv(url)

Unnamed: 0               0.000000
country                  0.000485
description              0.000000
designation              0.405001
points                   0.000000
price                    0.074362
province                 0.000485
region_1                 0.195421
region_2                 1.573123
taster_name              0.253010
taster_twitter_handle    0.316055
title                    0.000000
variety                  0.000008
winery                   0.000000
dtype: float64


In [ ]:
# Calcul du ratio entre les valeurs manquantes et le total des valeurs par colonne
ratio_missing_values = data.isna().sum() / data.count()

# Affichage du résultat
print(ratio_missing_values)

In [10]:
data['price'].fillna(data['price'].mean(), inplace=True)
data.describe()

,Unnamed: 0,points,price
count,129971.000000,129971.000000,129971.000000
mean,64985.000000,88.447138,35.363389
std,37519.540256,3.039730,39.577066
min,0.000000,80.000000,4.000000
25%,32492.500000,86.000000,18.000000
50%,64985.000000,88.000000,28.000000
75%,97477.500000,91.000000,40.000000
max,129970.000000,100.000000,3300.000000


In [11]:
ratio_missing_values = data.isna().sum() / data.count()

# Affichage du résultat
print(ratio_missing_values)

Unnamed: 0               0.000000
country                  0.000485
description              0.000000
designation              0.405001
points                   0.000000
price                    0.000000
province                 0.000485
region_1                 0.195421
region_2                 1.573123
taster_name              0.253010
taster_twitter_handle    0.316055
title                    0.000000
variety                  0.000008
winery                   0.000000
dtype: float64
